In [1]:
!pip install langchain langchain-openai langchain-chroma langchain-community langgraph beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Load the web page content using BeautifulSoup
bs4_strainer = bs4.SoupStrainer(class_=lambda x: x in ['post-title', 'post-content', 'post-header'])
loader = WebBaseLoader(
    web_paths=['https://lilianweng.github.io/posts/2023-06-23-agent/'],
    bs_kwargs={'parse_only': bs4_strainer}
)

docs = loader.load()
print(f'Loaded {len(docs)} documents.')
print(f'len(docs[0].page_content) = {len(docs[0].page_content)}')
print(docs[0].page_content[:100])

USER_AGENT environment variable not set, consider setting it to identify your requests.


Loaded 1 documents.
len(docs[0].page_content) = 43047


      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split the documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True
)

all_splits = text_splitter.split_documents(docs)

print(f'Created {len(all_splits)} splits.')
print(f'len(all_splits[0].page_content) = {len(all_splits[0].page_content)}')
print(all_splits[0].page_content)
print(f'len(all_splits[0].page_content) = {len(all_splits[0].page_content)}')
print(f'len(all_splits[1].page_content) = {len(all_splits[1].page_content)}')
print(f'len(all_splits[2].page_content) = {len(all_splits[2].page_content)}')

Created 63 splits.
len(all_splits[0].page_content) = 969
LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality 

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# Create a vector store from the split documents using OpenAI embeddings
vector_store = Chroma.from_documents(
    documents=all_splits,
    embedding=OpenAIEmbeddings(),
)

In [4]:
# Use the vector store as a retriever to find relevant documents

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke('What are the approaches to Task decomposition?')

print(len(retrieved_docs))

print(retrieved_docs[0].page_content)

6
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.
Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.


In [9]:
print(retrieved_docs[1].page_content)

Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
Another quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external classical planner to do long-horizon planning. This approach utilizes the Planning Domain Definition Language (PDDL) as an intermediate interface to describe the planning problem. In this process, LLM (1) translates the problem into “Problem PDDL”, then (2) requests a classical planner to generate a PDDL plan based on an existing “Domain PDDL”, and finally (3) translates the PDDL plan back into natural language. Essentially, the planning step is outsourced to an external tool, assuming the availability of domain-specific PDDL and a suitable planner which is common in certain robotic setups but not in many other domains.
Self-Reflection#


In [11]:
print(retrieved_docs[2].page_content)

Resources:
1. Internet access for searches and information gathering.
2. Long Term memory management.
3. GPT-3.5 powered Agents for delegation of simple tasks.
4. File output.

Performance Evaluation:
1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.
2. Constructively self-criticize your big-picture behavior constantly.
3. Reflect on past decisions and strategies to refine your approach.
4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.


In [ ]:
from langchain_openai import ChatOpenAI
from langchain import hub

# Initialize the LLM and pull the prompt from the hub

llm = ChatOpenAI(model="gpt-4o")
prompt = hub.pull('rlm/rag-prompt')


[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [7]:
# Format the retrieved documents for the prompt
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [8]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Build up the RAG chain with LCEL
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# Stream the response from the RAG chain
for chunk in rag_chain.stream('What are the approaches to Task decomposition?'):
    print(chunk, end='', flush=True)

The approaches to task decomposition include Chain of Thought (CoT), which involves breaking down complex tasks into smaller steps to enhance model performance, and Tree of Thoughts, which generates multiple reasoning paths at each step in a tree structure. Additionally, tasks can be decomposed using simple prompts, task-specific instructions, human inputs, or by relying on an external classical planner like LLM+P, which uses the Planning Domain Definition Language (PDDL) to describe and solve planning problems.